In [1]:
import numpy as np 
import os
from matplotlib import pyplot as plt 
import mediapipe as mp 
import cv2
import mediapipe as mp 



In [22]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [23]:
def mediapipe_detection(image, model): 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Color Conversion BGR 2 RGB 
    image.flags.writeable = False  #Image is no longer writeable 
    results = model.process(image) # Make predicition
    image.flags.writeable = True # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Color Conversion RGB 2 BGR 
    return image,results

In [24]:
def draw_landmarks(image, results): 
    # Draw Face Connections
    #The 4th parameter is going to draw the landmarks and the 5th the connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius = 1 ), 
                              mp_drawing.DrawingSpec(color=(80,256,51), thickness=1, circle_radius=1)) 
    # Draw Pose Connections 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius = 4 ), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius = 4 ), 
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius = 4 ), 
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                              ) 

In [25]:
# Extracting the keypoints 
def extract_keypoints(results): 
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)

    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
 
    return np.concatenate([pose,face,lh,rh])

In [3]:
dir_path = "../DataSet" 
actions = np.array(os.listdir(dir_path))


results_path = os.path.join('data_results')



for action in actions: 
    num_of_videos = os.listdir(dir_path+'/'+action)
    for video in range(len(num_of_videos)): 
        try: 
            os.makedirs(os.path.join(results_path, action, str(video)))
        except: 
            pass 


In [112]:
test_actions = ['Blue','Happy','Man', 'Family']
width_list = []

height_list = []
for action in test_actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)

    for file in range(num_files):
        file_name = file_list[file]
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)
        num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        width_list.append(width)
        height_list.append(height)

In [122]:
print(sum(height_list)/ len(height_list))

print(min(width_list))

907.0
1280


In [4]:
# holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) 
test_actions = ['Blue','Happy','Man', 'Family']

channels = 3
data = []
batch_size = 2
for action in test_actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)

    for file in range(num_files):
        file_name = file_list[file]
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)
        num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        # video_data =np.zeros((num_of_frames, height, width, 3 ), dtype= np.float32)
        rearrange_width = int(width/4)
        rearranged_height = int(height/4)
        # for i in range(0,num_of_frames,batch_size): 
        for i in range(0,num_of_frames,batch_size): 

            batch_data = np.zeros((num_of_frames, height, width, 3 ), dtype=np.float32)
            for k in range(batch_size): 
                ret, frame = cap.read()

                if not ret:
                    break 
                # frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                # frame = frame.astype(np.float32)/255.0

                batch_data[k] = frame
            data.append(batch_data)
            # del batch_data
        cap.release()

       
           

            # image, results = mediapipe_detection(frame, holistic)
            # cv2.imshow('',image)
            # print(results)

            # draw_landmarks(image,results)

            # keypoints = extract_keypoints(results)
            # npy_path = os.path.join(results_path, action, str(file), str(i) )
            # np.save(npy_path, keypoints)

# cap.release()
# cv2.destroyAllWindows()

# results = [120, frames_per_video, frame_height, frame_width,3 ]


MemoryError: Unable to allocate 422. MiB for an array with shape (30, 960, 1280, 3) and data type float32

In [130]:
frames = []
for action in actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)

    for file in range(num_files):
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)
        frames.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

max_frame = max(frames)

In [89]:
print(frames)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [131]:
channels = 3 
data =[]
for action in test_actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)

    for file in range(num_files):
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)
        frames_per_video = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print(frames_per_video)
        # video_data = np.zeros((max_frame, height, width, channels))
#         for frame in range(frames_per_video): 
#             ret, frame = cap.read()
#             if not ret: 
#                 break
#             video_data[frame] = frame 
#         data.append(video_data)
#         cap.release()
# data = np.array(data)



55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
55
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [72]:
results = np.array(results)
results.shape

c:\Users\amyan\anaconda3\envs\IPCV\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(5,)

## Training the model 

In [93]:
#Pre processing data and creating labels 
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


label_map = {label: num for num, label in enumerate(actions)}

In [97]:

sequences, labels = [], []

for action in actions: 
    file_list = os.listdir(dir_path+"/"+action)

    for video in range(len(file_list)): 
        # window = []
        # filename = file_list[video]
        # cap = cv2.VideoCapture(dir_path+"/"+action+"/"+filename)
        # num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # for frame_num in range(num_of_frames): 
        #     try: 
        #         res = np.load(os.path.join(results_path, action, str(video), "{}.npy".format(frame_num)))
        #         print(res)
        #         window.append(res)
        #         print("window")
        #         print(window)
        #     except FileNotFoundError: 
        #         print("File not found " + filename)
        #         continue
        # sequences.append(window)
        labels.append(label_map[action])

In [101]:
data.shape

(4,)

In [8]:
from keras_preprocessing.sequence import pad_sequences

In [100]:
# X = np.array(sequences)
padded_sequences = pad_sequences(data, padding='post', value=-999)


ValueError: Shape of sample (1080, 1920, 3) of sequence at position 2 is different from expected shape (960, 1280, 3)

In [53]:
input_shape = (padded_sequences.shape[0], padded_sequences.shape[1], padded_sequences.shape[2], 3,)

In [98]:
y= to_categorical(labels).astype(int)
y = np.array(y)

In [39]:
padded_sequences.shape

(120, 72, 1662)

In [99]:
x_train,x_test, y_train,y_test = train_test_split(data,y, test_size=0.1)

ValueError: Found input variables with inconsistent numbers of samples: [4, 120]

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense , Masking, Flatten, MaxPooling2D

In [62]:
model = Sequential()
# model.add(Masking(mask_value=-999, input_shape =input_shape))
model.add(Masking(mask_value=-999, input_shape =padded_sequences.shape))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu',input_shape =padded_sequences.shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [63]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [64]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_15 (Masking)         (None, 120, 72, 1662)     0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 118, 70, 32)       478688    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 59, 35, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 57, 33, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 16, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 28672)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)             

In [65]:
model.fit(x_train,y_train, epochs=2000)

Epoch 1/2000


ValueError: in user code:

    c:\Users\amyan\anaconda3\envs\IPCV\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    c:\Users\amyan\anaconda3\envs\IPCV\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\amyan\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\amyan\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\amyan\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\Users\amyan\anaconda3\envs\IPCV\lib\site-packages\keras\engine\